We will import the Necessary Libraries!

In [1]:
import pandas as pd
import numpy as np

Load the Dataset

In [2]:
movies_data = pd.read_csv('/content/movies.csv') # Load the dataset

In [3]:
movies_data.head(1)


,id,title,genres,original_language,overview,popularity,production_companies,release_date,budget,revenue,runtime,status,tagline,vote_average,vote_count,credits,keywords,poster_path,backdrop_path,recommendations
0,615656,Meg 2: The Trench,Action-Science Fiction-Horror,en,An exploratory dive into the deepest depths of...,8763.998,Apelles Entertainment-Warner Bros. Pictures-di...,2023-08-02,129000000.0,352056482.0,116.0,Released,Back for seconds.,7.079,1365.0,Jason Statham-Wu Jing-Shuya Sophia Cai-Sergio ...,based on novel or book-sequel-kaiju,/4m1Au3YkjqsxF8iwQy0fPYSxE0h.jpg,/qlxy8yo5bcgUw2KAmmojUKp4rHd.jpg,1006462-298618-569094-1061181-346698-1076487-6...


Feature Implimentation / Data Preproccessing

In [4]:
# Now we will select all the relevant features to give our cosine testing algorithm the best possible recommendation based on the user input
# genres,id,keywords,title,overview,credits
movies = movies_data[['id','title','overview','genres','keywords','credits']]

In [5]:
movies.head() # Now we have the full dataframe

,id,title,overview,genres,keywords,credits
0,615656,Meg 2: The Trench,An exploratory dive into the deepest depths of...,Action-Science Fiction-Horror,based on novel or book-sequel-kaiju,Jason Statham-Wu Jing-Shuya Sophia Cai-Sergio ...
1,758323,The Pope's Exorcist,Father Gabriele Amorth Chief Exorcist of the V...,Horror-Mystery-Thriller,spain-rome italy-vatican-pope-pig-possession-c...,Russell Crowe-Daniel Zovatto-Alex Essoe-Franco...
2,533535,Deadpool & Wolverine,A listless Wade Wilson toils away in civilian ...,Action-Comedy-Science Fiction,hero-superhero-anti hero-mutant-breaking the f...,Ryan Reynolds-Hugh Jackman-Emma Corrin-Matthew...
3,667538,Transformers: Rise of the Beasts,When a new threat capable of destroying the en...,Action-Adventure-Science Fiction,peru-alien-end of the world-based on cartoon-b...,Anthony Ramos-Dominique Fishback-Luna Lauren V...
4,693134,Dune: Part Two,Follow the mythic journey of Paul Atreides as ...,Science Fiction-Adventure,epic-based on novel or book-fight-sandstorm-sa...,Timothée Chalamet-Zendaya-Rebecca Ferguson-Jav...


Data Cleaning! Of the selected features, we will iterate through each column replace NULL values with 'NULL' strings..

In [6]:
movies.isnull().sum() # Check for null values

,0
id,0
title,6
overview,118243
genres,210317
keywords,511678
credits,224714


In [7]:
movies.dropna(inplace=True) # Drop the null values

<ipython-input-7-e526ab4ca26e>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  movies.dropna(inplace=True) # Drop the null values


In [8]:
movies.head() # Refresh the DataFrame

,id,title,overview,genres,keywords,credits
0,615656,Meg 2: The Trench,An exploratory dive into the deepest depths of...,Action-Science Fiction-Horror,based on novel or book-sequel-kaiju,Jason Statham-Wu Jing-Shuya Sophia Cai-Sergio ...
1,758323,The Pope's Exorcist,Father Gabriele Amorth Chief Exorcist of the V...,Horror-Mystery-Thriller,spain-rome italy-vatican-pope-pig-possession-c...,Russell Crowe-Daniel Zovatto-Alex Essoe-Franco...
2,533535,Deadpool & Wolverine,A listless Wade Wilson toils away in civilian ...,Action-Comedy-Science Fiction,hero-superhero-anti hero-mutant-breaking the f...,Ryan Reynolds-Hugh Jackman-Emma Corrin-Matthew...
3,667538,Transformers: Rise of the Beasts,When a new threat capable of destroying the en...,Action-Adventure-Science Fiction,peru-alien-end of the world-based on cartoon-b...,Anthony Ramos-Dominique Fishback-Luna Lauren V...
4,693134,Dune: Part Two,Follow the mythic journey of Paul Atreides as ...,Science Fiction-Adventure,epic-based on novel or book-fight-sandstorm-sa...,Timothée Chalamet-Zendaya-Rebecca Ferguson-Jav...


In [9]:
movies.isnull().sum() # Check again for count of null values

,0
id,0
title,0
overview,0
genres,0
keywords,0
credits,0


We want to turn each sentence into a list of csv's for our cosine similarity algorithm to further give a more active represenation of what movies should be recommended based on the user input.


In [10]:
for column in ['overview', 'genres', 'keywords', 'credits']:
    # Make each ith,jth entry a list of csv
    # Check if elements in the column are not already lists
    movies[column] = movies[column].apply(lambda x: x.split() if isinstance(x, str) else x)

<ipython-input-10-e8b6b2d0065c>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  movies[column] = movies[column].apply(lambda x: x.split() if isinstance(x, str) else x)


In [11]:
movies.head()

,id,title,overview,genres,keywords,credits
0,615656,Meg 2: The Trench,"[An, exploratory, dive, into, the, deepest, de...","[Action-Science, Fiction-Horror]","[based, on, novel, or, book-sequel-kaiju]","[Jason, Statham-Wu, Jing-Shuya, Sophia, Cai-Se..."
1,758323,The Pope's Exorcist,"[Father, Gabriele, Amorth, Chief, Exorcist, of...",[Horror-Mystery-Thriller],"[spain-rome, italy-vatican-pope-pig-possession...","[Russell, Crowe-Daniel, Zovatto-Alex, Essoe-Fr..."
2,533535,Deadpool & Wolverine,"[A, listless, Wade, Wilson, toils, away, in, c...","[Action-Comedy-Science, Fiction]","[hero-superhero-anti, hero-mutant-breaking, th...","[Ryan, Reynolds-Hugh, Jackman-Emma, Corrin-Mat..."
3,667538,Transformers: Rise of the Beasts,"[When, a, new, threat, capable, of, destroying...","[Action-Adventure-Science, Fiction]","[peru-alien-end, of, the, world-based, on, car...","[Anthony, Ramos-Dominique, Fishback-Luna, Laur..."
4,693134,Dune: Part Two,"[Follow, the, mythic, journey, of, Paul, Atrei...","[Science, Fiction-Adventure]","[epic-based, on, novel, or, book-fight-sandsto...","[Timothée, Chalamet-Zendaya-Rebecca, Ferguson-..."


We want to take credits column and also add the overview, genres, and keywords columns to present a more simpler df that we can work with.

In [12]:
movies['credits'] = movies['overview'] + movies['genres'] + movies['keywords']

<ipython-input-12-6f6e83054f15>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  movies['credits'] = movies['overview'] + movies['genres'] + movies['keywords']


In [13]:
movies.head()

,id,title,overview,genres,keywords,credits
0,615656,Meg 2: The Trench,"[An, exploratory, dive, into, the, deepest, de...","[Action-Science, Fiction-Horror]","[based, on, novel, or, book-sequel-kaiju]","[An, exploratory, dive, into, the, deepest, de..."
1,758323,The Pope's Exorcist,"[Father, Gabriele, Amorth, Chief, Exorcist, of...",[Horror-Mystery-Thriller],"[spain-rome, italy-vatican-pope-pig-possession...","[Father, Gabriele, Amorth, Chief, Exorcist, of..."
2,533535,Deadpool & Wolverine,"[A, listless, Wade, Wilson, toils, away, in, c...","[Action-Comedy-Science, Fiction]","[hero-superhero-anti, hero-mutant-breaking, th...","[A, listless, Wade, Wilson, toils, away, in, c..."
3,667538,Transformers: Rise of the Beasts,"[When, a, new, threat, capable, of, destroying...","[Action-Adventure-Science, Fiction]","[peru-alien-end, of, the, world-based, on, car...","[When, a, new, threat, capable, of, destroying..."
4,693134,Dune: Part Two,"[Follow, the, mythic, journey, of, Paul, Atrei...","[Science, Fiction-Adventure]","[epic-based, on, novel, or, book-fight-sandsto...","[Follow, the, mythic, journey, of, Paul, Atrei..."


Now we will create a new dataframe to create a more simpler visual!

In [14]:
new_df = movies[['id','title','credits']]
new_df

,id,title,credits
0,615656,Meg 2: The Trench,"[An, exploratory, dive, into, the, deepest, de..."
1,758323,The Pope's Exorcist,"[Father, Gabriele, Amorth, Chief, Exorcist, of..."
2,533535,Deadpool & Wolverine,"[A, listless, Wade, Wilson, toils, away, in, c..."
3,667538,Transformers: Rise of the Beasts,"[When, a, new, threat, capable, of, destroying..."
4,693134,Dune: Part Two,"[Follow, the, mythic, journey, of, Paul, Atrei..."
...,...,...,...
722271,724660,Searching for Nepal,"[A, former, Peace, Corps, volunteer, returns, ..."
722286,806445,Mythical Creatures - A Documentary About the B...,"[Renowned, Artist, Gary, Baseman, embarks, on,..."
722308,914660,A Hundred and Seventy Seconds,"[Usually, we, don't, stop, and, analyze, the, ..."
722313,527223,Strictly Modern,"[Strictly, Modern, is, a, 1930, American, pre-..."


In [15]:
new_df['credits'] = new_df['credits'].apply(lambda x: " ".join(x))

<ipython-input-15-2aeb52b1cd1c>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  new_df['credits'] = new_df['credits'].apply(lambda x: " ".join(x))


In [16]:
new_df.head()

,id,title,credits
0,615656,Meg 2: The Trench,An exploratory dive into the deepest depths of...
1,758323,The Pope's Exorcist,Father Gabriele Amorth Chief Exorcist of the V...
2,533535,Deadpool & Wolverine,A listless Wade Wilson toils away in civilian ...
3,667538,Transformers: Rise of the Beasts,When a new threat capable of destroying the en...
4,693134,Dune: Part Two,Follow the mythic journey of Paul Atreides as ...


In [17]:
new_df['credits'][0]

'An exploratory dive into the deepest depths of the ocean of a daring research team spirals into chaos when a malevolent mining operation threatens their mission and forces them into a high-stakes battle for survival. Action-Science Fiction-Horror based on novel or book-sequel-kaiju'

In [18]:
new_df['credits'] = new_df['credits'].apply(lambda x:x.lower())

<ipython-input-18-b254f47f7b04>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  new_df['credits'] = new_df['credits'].apply(lambda x:x.lower())


In [19]:
new_df.head()

,id,title,credits
0,615656,Meg 2: The Trench,an exploratory dive into the deepest depths of...
1,758323,The Pope's Exorcist,father gabriele amorth chief exorcist of the v...
2,533535,Deadpool & Wolverine,a listless wade wilson toils away in civilian ...
3,667538,Transformers: Rise of the Beasts,when a new threat capable of destroying the en...
4,693134,Dune: Part Two,follow the mythic journey of paul atreides as ...


In [20]:
new_df['credits'][0]

'an exploratory dive into the deepest depths of the ocean of a daring research team spirals into chaos when a malevolent mining operation threatens their mission and forces them into a high-stakes battle for survival. action-science fiction-horror based on novel or book-sequel-kaiju'

We will now begin the Vectorization Process!

In [21]:
# max_features=5000: This limits the number of words (features) to consider to the 5000 ''MOST FREQUENT WORDS'' in the text data.
# This helps reduce the dimensionality of the data and can improve performance.
# stop_words='english': This tells CountVectorizer to remove common English words like "the," "a," "is," etc., which are often not very informative for analysis. By removing these, we focus on the more meaningful words in the text.
from sklearn.feature_extraction.text import CountVectorizer
cv = CountVectorizer(max_features=5000,stop_words='english')

In [22]:
vectors = cv.fit_transform(new_df['credits']).toarray()

In [23]:
vectors

array([[0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       ...,
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0]])

In [24]:
feature_names = cv.get_feature_names_out()
feature_names

array(['10', '100', '11', ..., 'zombies', 'zone', 'zoo'], dtype=object)

In [25]:
import nltk
from nltk.stem.porter import PorterStemmer
ps = PorterStemmer()

In [26]:
def stem(text):
  y = []
  for i in text.split():
    y.append(ps.stem(i)) # Pass the loop variable 'i' (word) to ps.stem()
  return " ".join(y)

In [27]:
ps.stem('loving') # Take similar words like loving, lovely, etc.. and converts to a base word love

'love'

In [28]:
new_df['credits'] = new_df['credits'].apply(stem)

<ipython-input-28-3745591ed2e6>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  new_df['credits'] = new_df['credits'].apply(stem)


In [29]:
from sklearn.metrics.pairwise import cosine_similarity
# Takes key data from genres, keywords, credits etc.. and converts them to decimal vals and the cosine similarity algorithm takes the simlarity value of the user input and compares
# them to other movies that have a similar similarity values


In [ ]:
similarity = cosine_similarity(vectors)

In [ ]:
def recommend():
    movie = input("Enter your favorite movie title: ")  # Get user input for the movie title

    try:
        # Find the index of the movie in the DataFrame
        index = new_df[new_df['title'] == movie].index[0]

        # Get the similarity distances for that movie
        distances = similarity[index]

        # Sort the distances and get the indices of the top 10 similar movies in descending order of similarity value implying that the top movie will be the most similar
        movies_list = sorted(enumerate(distances), key=lambda x: x[1], reverse=True)[1:11]

        # Get the titles of the recommended movies
        recommended_movies = [new_df.iloc[i[0]].title for i in movies_list]

        # Print the recommended movies
        print("\nRecommended movies:")
        for title in recommended_movies:
            print(title)
    except IndexError:
        print("Sorry, the movie title you entered is not found. Please try entering another keyword!")

# Call the function to test it
recommend()
